<a href="https://colab.research.google.com/github/Guilherme-Cosso/COREM/blob/main/AS05_Implementa%C3%A7%C3%A3o_de_Assistente_Conversacional_Baseado_em_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AS05: Implementação de Assistente Conversacional Baseado em LLM
##TAREFA
Esta é a tarefa AS05: Implementação de Assistente Conversacional Baseado em LLM, uma atividade prática que estimula o aluno a absorver conceitos básicos de mineração e análise de texto.

## Problema
Implementar um assistente conversacional baseado em LLM que seja capaz de indexar vetores (embeddings textuais) de uma coleção de documentos PDF para posteriormente responder a perguntas feitas através de uma interfce de conversação.

### Instale tudo na sua maquina

In [ ]:
!pip install transformers

In [ ]:
pip install PyPDF2

In [ ]:
pip install transformers

### Pre-Implementação
Desenvolvi um código para garantir a criação da pasta 'pdfs' em seu diretório local, visando evitar possíveis falhas. Todos os arquivos PDF presentes nessa pasta serão considerados durante a geração do modelo de conversação baseado em LLM.

Para importar os PDFs gerados no modelo, sugiro duas abordagens:

**Modelo 1: Criação manual da pasta 'pdfs':**

Utilize o primeiro modelo, o qual apenas cria a pasta 'pdfs' no seu diretório local. Você poderá importar manualmente os arquivos a serem utilizados para o treinamento.

**Modelo 2: Importação automática de arquivos:**

Opte pelo segundo modelo, que permite importar quantos arquivos desejar. Esse modelo criará automaticamente a pasta 'pdfs' e a preencherá com os arquivos importados passados pelo usuario.

In [ ]:
#Modelo 1: Criação manual da pasta 'pdfs'
import os
nome_da_pasta = 'pdfs'
# Verifica se a pasta não existe antes de criar para evitar erros
if not os.path.exists(nome_da_pasta):
    os.makedirs(nome_da_pasta)
    print(f'A pasta "{nome_da_pasta}" foi criada.')
else:
    print(f'A pasta "{nome_da_pasta}" já existe.')

In [12]:
# Modelo 2: Importação automática de arquivos:
from google.colab import files
import shutil
import os

def import_Arq():
  arquivos = os.listdir('/content')
  nome_da_pasta = 'pdfs'
  for arquivo in arquivos:
      if arquivo.endswith('.pdf'):
          if not os.path.exists(nome_da_pasta):
              os.makedirs(nome_da_pasta)
          shutil.move(os.path.join('/content', arquivo), os.path.join(nome_da_pasta, arquivo))

entrada_usuario = '1'
while entrada_usuario == '1':
    upload = files.upload()  # Pede para o usuário fazer upload de um arquivo
    # Pergunta se o usuário deseja inserir mais um arquivo
    entrada_usuario = input("Deseja inserir mais um arquivo? \n(1) Sim\n(0) Não\n")
    # Certifica-se de que a entrada é válida (1 ou 0)
    while entrada_usuario not in ['0', '1']:
        entrada_usuario = input("Por favor, escolha 1 para 'Sim' ou 0 para 'Não': \n")
    if entrada_usuario == '0':
        print("Preparendo o processo de inserção de arquivos.")
        import_Arq();

Saving Platão-O-Político.pdf to Platão-O-Político.pdf
Deseja inserir mais um arquivo? 
(1) Sim
(0) Não
0
Preparendo o processo de inserção de arquivos.


### Data base

Agora que todos os arquivos ja foram inseridos iremos realizar uma leitura textual de todos que estão na pasta 'pdfs'.Normalizar os dados e colocalos no arquivo "all.txt".

**Normalização**

In [6]:
import re
def normalization(conteudo):
  # Tirar numeros
  conteudo = re.sub(r'-?\d+(?:\.\d+)?', ' ', conteudo)
  return conteudo;

**Escrita em Arquivo**

In [7]:
def write(conteudo, nome_arq):
  # Nome do arquivo que você quer criar
  nome_arquivo = nome_arq +'.txt'
  with open(nome_arquivo, 'w') as arquivo:
      arquivo.write(conteudo)

**Leitura dos Pdfs**

In [15]:
# Abre o arquivo PDF no modo leitura binária ('rb')
import os
import PyPDF2

# Diretório onde estão os arquivos PDF
diretorio_pdfs = '/content/pdfs/'
text ='';
# Percorre todos os arquivos na pasta pdfs
for filename in os.listdir(diretorio_pdfs):
    if filename.endswith('.pdf'):  # Verifica se é um arquivo PDF
        file_path = os.path.join(diretorio_pdfs, filename)

        # Abre o arquivo PDF no modo leitura binária ('rb')
        with open(file_path, 'rb') as pdf_file:
            # Cria um objeto PDF Reader
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            # Extrai o texto de todas as páginas
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
text = normalization(text)
write(text, "all")

###Tokenização

Realizamos a etapa de tokenizaçao on tokenizamos as palavras presentes no aquivo all.txt onde foi colacado todos os Pdf extraidos.


In [ ]:
from tokenizers import ByteLevelBPETokenizer
PATH = './sample_data/'
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=['all.txt'], vocab_size=52000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"
])
if not os.path.exists(PATH+"model"):
  os.makedirs(PATH+"model")
tokenizer.save_model(PATH+"model")


### Configuração da Roberta

Para garantir que os tokens gerados pelo ByteLevelBPETokenizer sejam utilizados corretamente pela RobertaTokenizer, verifique se os tokens seguem o formato esperado e utilize a função RobertaTokenizer.encode para converter os textos em sequências de IDs de token compatíveis com os modelos baseados na arquitetura ROBERTA.

In [19]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained(PATH+'model', max_len=512)

**Configurações**

In [20]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=512,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

**Parametros da REDE NEURAL**

In [ ]:
model.num_parameters()

In [28]:
#forma simples de se carregar um arquivo bruto como Dataset
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="all.txt",
    block_size=128,
)


In [24]:
'''
Data Collators são estratégias de se construir lotes de dados
para treinar o modelo. Cria listas de amostras a partir do dataset e
permite que o Pytorch aplique a backpropagation adequadamente.
Probability = probabilidade de mascarar tokens da entrada
'''
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.1
)


In [25]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=PATH+'model',
    overwrite_output_dir=True,
    num_train_epochs=1200,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

**Treino e salvo o modelo**

In [26]:
trainer.train()

Step,Training Loss
500,6.563800
1000,5.263400
1500,4.273400
2000,3.286300
2500,2.433800
3000,1.761500
3500,1.274900
4000,0.951000
4500,0.774300


TrainOutput(global_step=4800, training_loss=2.8133798853556313, metrics={'train_runtime': 5211.2182, 'train_samples_per_second': 52.042, 'train_steps_per_second': 0.921, 'total_flos': 8992119855513600.0, 'train_loss': 2.8133798853556313, 'epoch': 1200.0})

In [27]:
trainer.save_model(PATH+'model')

**Testo ele**

In [30]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=PATH+'model',
    tokenizer=PATH+'model'
)


In [37]:
pergunta = 'sócrates é uma <mask>'
fill_mask(pergunta)

[{'score': 0.07704049348831177,
  'token': 725,
  'token_str': ' sócrates',
  'sequence': 'sócrates é uma sócrates'},
 {'score': 0.04383159801363945,
  'token': 16,
  'token_str': ',',
  'sequence': 'sócrates é uma,'},
 {'score': 0.03124419040977955,
  'token': 18,
  'token_str': '.',
  'sequence': 'sócrates é uma.'},
 {'score': 0.023870207369327545,
  'token': 6,
  'token_str': '"',
  'sequence': 'sócrates é uma"'},
 {'score': 0.01627921685576439,
  'token': 1602,
  'token_str': ' causalidade',
  'sequence': 'sócrates é uma causalidade'}]